# Theory: Anthropic API & Gradio for a 20 Questions Game

### Gradio Interface
Gradio is used to create an interactive web interface for the game.
- **`gr.ChatInterface`**: This is a high-level Gradio component that quickly creates a chatbot UI.
  ```python
  demo = gr.ChatInterface(
      predict, # The function to call when a user sends a message
      examples=["Start!"], # Example inputs
      chatbot=gr.Chatbot(placeholder=placeholder), # Customizing the chatbot display
      type="messages" # Using the newer messages format for history
  )
  # demo.launch() # Called in the code cell
  ```
- **`predict` function**: This function is the heart of the Gradio app. It takes the user's `message` and the `history` of the conversation.
    - It appends the new user message to the history.
    - It calls the Anthropic API.
    - It returns the AI's response. The example code returns a dictionary with `role`, `content`, and `options` for quick replies (Yes/No buttons), which `gr.Chatbot` can process.
- **`gr.Chatbot` and `placeholder`**: The `placeholder` provides initial instructions or a welcome message in the chat window.

### 3. Game Logic
- **History Management**: The `predict` function ensures the conversation `history` is correctly formatted (as a list of message dictionaries for `type="messages"`) and passed to the Anthropic API.
- **System Prompt**: This is key to guiding the AI's behavior. The example script instructs the AI to play 10 questions, ask yes/no questions, and respond in Hebrew.

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
import anthropic
import gradio as gr

client = anthropic.Anthropic(api_key=api_key)

def predict(message: str, history: list):
    if client is None:
        return "Error: Anthropic client not initialized due to missing API key."
        
    # For type="messages", history is a list of dicts: [{'role': 'user', 'content': '...'}, ...]
    keys_to_keep = ["role", "content"]
    history = [{k: d[k] for k in keys_to_keep if k in d} for d in history]
    
    history.append({"role": "user", "content": message})
    
    # Limit the game to roughly 10 questions from the AI (plus user turns)
    # The system prompt also guides the AI on the number of questions.
    if len(history) > 20: 
        history.append({"role": "user", "content": "DONE"})
        
    output = client.messages.create(
        messages=history,  
        model="claude-sonnet-4-20250514", # User's specified model
        max_tokens=1000,
        system="You are guessing an object that the user is thinking of. You can ask 10 yes/no questions. Keep asking questions until the user says DONE, speak hebrew"
    )
        
    # For gr.ChatInterface with type="messages", the function can return the assistant's message string directly.
    # However, to support custom options like Yes/No buttons, returning a dict that gr.Chatbot can process is a valid pattern if the Chatbot is configured to handle it.
    return {
        "role": "assistant",
        "content": output.content[0].text,  
        "options": [{"value": "Yes"}, {"value": "No"}]
    }

placeholder = """
<center><h1>10 Questions</h1><br>Think of a person, place, or thing. I'll ask you 10 yes/no questions to try and guess it.
</center>
"""

demo = gr.ChatInterface(
    predict,
    examples=["Start!"],
    chatbot=gr.Chatbot(placeholder=placeholder),
    type="messages" 
)

demo.launch()

C:\Users\eyalra\AppData\Local\Temp\ipykernel_30072\4197186709.py:44: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(placeholder=placeholder),
c:\Users\eyalra\AppData\Local\miniconda3\envs\genaienv\Lib\site-packages\gradio\chat_interface.py:314: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


: 

## Beginner Exercise

Here's a simple task to get you started with modifying the game:

**Task**: Change the AI's persona and the game's theme.

1.  **Modify the `system` prompt**:
    In the Python code cell above, find the line where the `system` prompt is defined within the `predict` function's call to `client.messages.create()`:
    ```python
    system="You are guessing an object that the user is thinking of. You can ask 10 yes/no questions. Keep asking questions until the user says DONE, speak hebrew"
    ```
    Change this text to something different. For example:
    *   Make the AI a space explorer guessing a planet: `"You are a curious space explorer. The user is thinking of a planet in our solar system. Ask yes/no questions to guess it. Keep your responses short and inquisitive."`
    *   Make the AI a food critic guessing a dish: `"You are a renowned food critic with a refined palate. The user is thinking of a famous dish. Ask yes/no questions to identify it. Be a little bit snarky but polite."`

2.  **Adjust the `placeholder` (Optional)**:
    You might want to update the `placeholder` text assigned to the `gr.Chatbot` to match your new theme (defined in the code cell above).

3.  **Run the code cell** and see how the AI's behavior changes when you interact with the Gradio interface!

**Bonus Questions for Thought**:
*   How does changing `max_tokens` affect the game?
*   What happens if you remove the "speak hebrew" part from the original system prompt?
*   Can you modify the `examples` in `gr.ChatInterface` to better suit your new theme?

In [ ]:
# fill code